In [6]:
import pandas as pd

In [7]:
data_path = '/data_volume/memento/lupus/ldsc/'

In [8]:
cts =['T8', 'T4', 'NK', 'ncM', 'cM', 'B']


### Read the gene loc file

In [9]:
gene_loc = pd.read_table(data_path + 'resources/gene_locs.txt')

In [10]:
# Add 100k window
gene_loc['start'] = (gene_loc['start'] - 100000).clip(lower=0)
gene_loc['end'] = gene_loc['end'] + 100000

### Get egenes and make bedfiles for memento

In [15]:
for pop in ['eur', 'asian','both']:
    for ct in cts:
        print(pop, ct)
        if pop == 'both':
            result1 = pd.read_csv('/data_volume/memento/lupus/full_analysis/memento/100kb/{}_{}.csv'.format('eur', ct)).query('FDR < 0.05')
            result2 = pd.read_csv('/data_volume/memento/lupus/full_analysis/memento/100kb/{}_{}.csv'.format('asian', ct)).query('FDR < 0.05')
            combined = result1.merge(result2, on=['SNP', 'gene'])
            combined[['gene']].drop_duplicates().to_csv(data_path + 'egenes/memento/{}_{}.txt'.format(pop, ct),index=False, header=False, sep='\t')
            print('overlap', combined.shape[0], result1.shape[0], result2.shape[0])
        else:
            result = pd.read_csv('/data_volume/memento/lupus/full_analysis/memento/100kb/{}_{}.csv'.format(pop, ct))
            result.query('FDR < 0.05')[['gene']].drop_duplicates().to_csv(data_path + 'egenes/memento/{}_{}.txt'.format(pop, ct),index=False, header=False, sep='\t')        

eur T8
eur T4
eur NK
eur ncM
eur cM
eur B
asian T8
asian T4
asian NK
asian ncM
asian cM
asian B
both T8
overlap 7586 21554 36615
both T4
overlap 11551 43468 33261
both NK
overlap 5189 15947 15377
both ncM
overlap 8971 26306 26349
both cM
overlap 23383 65487 66853
both B
overlap 7410 21447 23697


In [8]:
for pop in ['eur', 'asian', 'both']:
    for ct in cts:
        egenes = pd.read_table(data_path + 'egenes/memento/{}_{}.txt'.format(pop, ct), header=None).iloc[:, 0].tolist()
        gene_loc.query('name in @egenes')[['chr', 'start', 'end', 'name']].to_csv(data_path + 'bedfiles/memento/{}_{}.bed'.format(pop, ct), index=False, header=False, sep='\t')

### Get egenes and make bedfiles for matrix eqtl

In [14]:
for pop in ['both']:
    for ct in cts:
        print(pop, ct)
        if pop == 'both':
            result1 = pd.read_table('/data_volume/memento/lupus/full_analysis/mateqtl/outputs/{}_{}_all_hg19.csv'.format('eur', ct)).query('FDR < 0.05')
            result2 = pd.read_table('/data_volume/memento/lupus/full_analysis/mateqtl/outputs/{}_{}_all_hg19.csv'.format('asian', ct)).query('FDR < 0.05')
            combined = result1.merge(result2, on=['SNP', 'gene'])
            combined[['gene']].drop_duplicates().to_csv(data_path + 'egenes/mateqtl/{}_{}.txt'.format(pop, ct),index=False, header=False, sep='\t')
            print('overlap', combined.shape[0], result1.shape[0], result2.shape[0])

        else:
            result = pd.read_table('/data_volume/memento/lupus/full_analysis/mateqtl/outputs/{}_{}_all_hg19.csv'.format(pop, ct))
            result.query('FDR < 0.05')[['gene']].drop_duplicates().to_csv(data_path + 'egenes/mateqtl/{}_{}.txt'.format(pop, ct),index=False, header=False, sep='\t')        

both T8
overlap 3039 6542 10536
both T4
overlap 3916 14369 10396
both NK
overlap 844 3357 2823
both ncM
overlap 3849 10413 10072
both cM
overlap 9142 24867 27078
both B
overlap 1519 3283 6168


In [12]:
cts =['T8', 'T4', 'NK', 'ncM', 'cM', 'B']
for pop in ['eur', 'asian', 'both']:
    for ct in cts:
        egenes = pd.read_table(data_path + 'egenes/mateqtl/{}_{}.txt'.format(pop, ct), header=None).iloc[:, 0].tolist()
        gene_loc.query('name in @egenes')[['chr', 'start', 'end', 'name']].to_csv(data_path + 'bedfiles/mateqtl/{}_{}.bed'.format(pop, ct), index=False, header=False, sep='\t')

### Get "egenes" and make bedfile for all expressed genes

In [16]:
all_genes = set()
for pop in ['asian', 'eur']:
    for ct in cts:
        result = pd.read_csv('/data_volume/memento/lupus/full_analysis/memento/100kb/{}_{}.csv'.format(pop, ct))
        all_genes |= set(result['gene'])
gene_loc.query('name in @all_genes')[['chr', 'start', 'end', 'name']].to_csv(data_path + 'bedfiles/pbmc.bed'.format(pop, ct), index=False, header=False, sep='\t')